In [1]:
"""Run a zero-inflated GP on opioid data"""
import os
import sys
from glob import glob

import numpy as np
import pandas as pd
idx = pd.IndexSlice
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import copy

import geopandas as gpd

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

import gpflow
import tensorflow as tf

from math import radians, cos, sin, asin, sqrt

code_dir = '/cluster/home/kheuto01/code/zero-inflated-gp/'
sys.path.append(code_dir)
from onoffgpf import OnOffSVGP, OnOffLikelihood

import pickle

/cluster/tufts/hugheslab/kheuto01/miniconda3/envs/zigp_36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/cluster/tufts/hugheslab/kheuto01/miniconda3/envs/zigp_36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/cluster/tufts/hugheslab/kheuto01/miniconda3/envs/zigp_36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("q

In [2]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)
    https://stackoverflow.com/a/4913653/1748679
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r


def top_X(y_true, y_pred, X=10):
    top_X_predicted = y_pred.sort_values(ascending=False)[:X]
    top_X_true = y_true.sort_values(ascending=False)[:X]

    undisputed_top_predicted = top_X_predicted[top_X_predicted > top_X_predicted.min()]
    num_tied_spots = X - len(undisputed_top_predicted)
    undisputed_top_true = top_X_true[top_X_true > top_X_true.min()]
    num_true_ties = X - len(undisputed_top_true)

    tied_top_predicted = top_X_predicted[top_X_predicted == top_X_predicted.min()]
    tied_top_true = top_X_true[top_X_true == top_X_true.min()]

    error_in_top_true_ties = np.abs(tied_top_true - y_pred[tied_top_true.index]).sort_values(ascending=True)
    error_in_top_pred_ties = np.abs(y_true[tied_top_predicted.index] - tied_top_predicted).sort_values(ascending=True)
    top_true_tied_geoids = error_in_top_true_ties[:num_true_ties].index
    top_pred_tied_geoids = error_in_top_pred_ties[:num_tied_spots].index

    best_possible_top_true_geoids = pd.Index.union(undisputed_top_true.index, top_true_tied_geoids)
    best_possible_top_pred_geoids = pd.Index.union(undisputed_top_predicted.index, top_pred_tied_geoids)

    # True values of GEOIDS with highest actual deaths. If ties, finds tied locations that match preds best
    best_possible_true = y_true[best_possible_top_true_geoids]
    best_possible_pred = y_true[best_possible_top_pred_geoids]

    assert (len(best_possible_true) == X)
    assert (len(best_possible_pred) == X)

    best_possible_absolute = np.abs(best_possible_true.sum() - best_possible_pred.sum())
    best_possible_ratio = np.abs(best_possible_pred).sum() / np.abs(best_possible_true).sum()

    bootstrapped_tied_indices = np.random.choice(tied_top_predicted.index, (1000, num_tied_spots))
    bootstrapped_all_indices = [pd.Index.union(undisputed_top_predicted.index,
                                               bootstrap_index) for bootstrap_index in bootstrapped_tied_indices]

    bootstrapped_absolute = np.mean([np.abs(top_X_true.sum() - y_true[indices].sum())
                                     for indices in bootstrapped_all_indices])
    bootstrapped_ratio = np.mean([np.abs(y_true[indices]).sum() / np.abs(top_X_true).sum()
                                  for indices in bootstrapped_all_indices])

    return best_possible_absolute, best_possible_ratio, bootstrapped_absolute, bootstrapped_ratio

In [3]:
time='qtr'
data_dir='/cluster/tufts/hugheslab/datasets/NSF_OD/'
inducing_points=3
iterations=10
out_dir='/cluster/home/kheuto01/test/'
auto_kernel=False

In [4]:
result_dir = os.path.join(data_dir, 'results')
mass_shapefile = os.path.join(data_dir, 'shapefiles', 'MA_2021')

svi_file = os.path.join(result_dir, 'svi_month')
svi_gdf = gpd.read_file(svi_file)
# Call it "grid_squar" because geopandas only supports len 10 columns
svi_gdf = svi_gdf.rename(columns={'INTPTLAT': 'lat', 'INTPTLON': 'lon', 'GEOID': 'grid_squar'})
# Make lat and lon floats
svi_gdf.loc[:, 'lat'] = svi_gdf.lat.astype(float)
svi_gdf.loc[:, 'lon'] = svi_gdf.lon.astype(float)
deaths_gdf = svi_gdf

# Used when we just need the unique tracts and their locations
just_grid = deaths_gdf.loc[
    (deaths_gdf['year'] == 2000) & (deaths_gdf['month'] == 1), ['grid_squar', 'geometry', 'lat', 'lon']]

# Calculate each squares neighbors
neighbors = {}
for _, row in just_grid.iterrows():
    just_grid.loc[:, 'haversine'] = just_grid.apply(lambda x: haversine(row['lon'], row['lat'],
                                                                        x['lon'], x['lat']),
                                                    axis=1)
    matching_neighbors = just_grid[just_grid['haversine'] < 8]['grid_squar'].values
    neighbors[row['grid_squar']] = matching_neighbors

tracts = deaths_gdf['grid_squar'].unique()
min_year = deaths_gdf.year.min()
max_year = deaths_gdf.year.max()
deaths_gdf = deaths_gdf.set_index(['grid_squar', 'year', 'month']).sort_index()

month_since_2000 = 0
season_since_2000 = 0
qtr_since_2000 = 0
year_since_2000 = 0
for year in range(min_year, max_year + 1):
    for month in range(1, 12 + 1):

        if month in [1, 2, 3, 4, 5, 6]:
            season = 'jan-jun'
        else:
            season = 'jul-dec'

        if month <= 3:
            qtr = 1
        elif month <= 6:
            qtr = 2
        elif month <= 9:
            qtr = 3
        else:
            qtr = 4

        deaths_gdf.loc[idx[:, year, month], 'month_since_2000'] = month_since_2000
        deaths_gdf.loc[idx[:, year, month], 'season'] = season
        deaths_gdf.loc[idx[:, year, month], 'season_since_2000'] = season_since_2000
        deaths_gdf.loc[idx[:, year, month], 'qtr'] = qtr
        deaths_gdf.loc[idx[:, year, month], 'qtr_since_2000'] = qtr_since_2000
        deaths_gdf.loc[idx[:, year, month], 'year_since_2000'] = year_since_2000

        month_since_2000 += 1

        if month in [6, 12]:
            season_since_2000 += 1

        if month in [3, 6, 9, 12]:
            qtr_since_2000 += 1

        if month == 12:
            year_since_2000 += 1

deaths_gdf = deaths_gdf.reset_index()
tracts = deaths_gdf['grid_squar'].unique()
min_year = deaths_gdf.year.min()
max_year = deaths_gdf.year.max()
deaths_gdf = deaths_gdf.set_index(['grid_squar', 'year', 'month']).sort_index()
deaths_gdf.loc[idx[:, :, :], 'last_timestep'] = deaths_gdf.loc[idx[:, :, :], 'deaths'].shift(1, )
deaths_gdf.loc[idx[:, :, :], 'last_year'] = deaths_gdf.loc[idx[:, :, :], 'deaths'].shift(12)
deaths_gdf.loc[idx[:, :, :], 'delta_deaths'] = deaths_gdf.loc[idx[:, :, :], 'deaths'] - deaths_gdf.loc[
    idx[:, :, :], 'last_timestep']
for tract in tracts:
    deaths_gdf.loc[idx[tract, :, :], 'neighbors_last_timestep'] = \
        deaths_gdf.loc[idx[neighbors[tract], :, :], 'last_timestep'].groupby(level=['year', 'month']).mean().shift(
            1).values
    deaths_gdf.loc[idx[tract, :, :], 'neighbors_last_year'] = \
        deaths_gdf.loc[idx[neighbors[tract], :, :], 'last_year'].groupby(level=['year', 'month']).mean().shift(
            12).values

deaths_gdf = deaths_gdf.reset_index()

In [5]:
old_deaths = copy.deepcopy(deaths_gdf)

In [6]:
    if time=='qtr':
        timestep_col = 'qtr_since_2000'
        deaths_gdf_meta = deaths_gdf.groupby(['grid_squar', 'year', timestep_col]).mean()[
            ['theme_3_pc', 'lon', 'theme_2_pc', 'lat', 'svi_pctile', 'theme_1_pc', 'theme_4_pc']]
        deaths_gdf = deaths_gdf.groupby(['grid_squar', 'year', timestep_col]).sum(min_count=3)[
            ['deaths', 'delta_deaths', 'last_timestep', 'last_year', 'neighbors_last_timestep', 'neighbors_last_year']]

        deaths_gdf = deaths_gdf.merge(deaths_gdf_meta, left_index=True, right_index=True)
        deaths_gdf.loc[idx[:, :, :], 'last_timestep'] = deaths_gdf.loc[idx[:, :, :], 'deaths'].shift(1, )
        deaths_gdf.loc[idx[:, :, :], 'last_year'] = deaths_gdf.loc[idx[:, :, :], 'deaths'].shift(2)
        deaths_gdf.loc[idx[:, :, :], 'delta_deaths'] = deaths_gdf.loc[idx[:, :, :], 'deaths'] - \
                                                       deaths_gdf.loc[idx[:, :], 'last_timestep']
        for tract in tracts:
            deaths_gdf.loc[idx[tract, :, :], 'neighbors_last_timestep'] = \
                deaths_gdf.loc[idx[neighbors[tract], :, :], 'last_timestep'].groupby(
                    level=[timestep_col, 'year']).mean().shift(1).values
            deaths_gdf.loc[idx[tract, :, :], 'neighbors_last_year'] = \
                deaths_gdf.loc[idx[neighbors[tract], :, :], 'last_year'].groupby(
                    level=[timestep_col, 'year']).mean().shift(4).values

In [7]:
deaths_gdf_with_autoregressive = deaths_gdf.reset_index()
if auto_kernel:
    features = ['grid_squar', 'lat', 'lon', timestep_col, 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
     'svi_pctile', 'neighbors_last_timestep', 'last_timestep']
    features_no_idx = ['lat', 'lon', timestep_col, 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
     'svi_pctile', 'neighbors_last_timestep', 'last_timestep']
else:
    features = ['grid_squar', 'lat', 'lon', timestep_col, 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
     'svi_pctile']
    features_no_idx = ['lat', 'lon', timestep_col, 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
     'svi_pctile']
    
train_x_through_2018 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year'] <= 2018][features].dropna()
train_y_through_2018 = deaths_gdf_with_autoregressive.loc[train_x_through_2018.index][
    ['grid_squar', timestep_col, 'deaths']].dropna()
train_x_through_2019 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year'] <= 2019][
    features].dropna()
train_y_through_2019 = deaths_gdf_with_autoregressive.loc[train_x_through_2019.index][
    ['grid_squar', timestep_col, 'deaths']].dropna()

x_just_2019 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year'] == 2019][
    features]
y_just_2019 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year'] == 2019][
    ['grid_squar', timestep_col, 'deaths']]
x_just_2020 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year'] == 2020][
    features]
y_just_2020 = deaths_gdf_with_autoregressive[deaths_gdf_with_autoregressive['year'] == 2020][
    ['grid_squar', timestep_col, 'deaths']]

spatial_kernel = gpflow.kernels.RBF(2, active_dims=[0, 1])
temporal_kernel = gpflow.kernels.RBF(1, active_dims=[2])
demo_kernel = gpflow.kernels.RBF(5, active_dims=[3, 4, 5, 6, 7])

if auto_kernel:
    autoregressive_kernel = gpflow.kernels.RBF(2, active_dims=[8,9])
    gaussian_kernel = spatial_kernel + temporal_kernel + demo_kernel + autoregressive_kernel
else:
    gaussian_kernel = spatial_kernel + temporal_kernel + demo_kernel

f_kernel = copy.deepcopy(gaussian_kernel)
g_kernel = copy.deepcopy(gaussian_kernel)
likelihood = OnOffLikelihood()

random = np.random.default_rng(seed=1)

M = inducing_points

N = len(train_x_through_2018)
Z = random.choice(train_x_through_2018[
                      features_no_idx].values, size=M, replace=False)

Zf = copy.deepcopy(Z)
Zg = copy.deepcopy(Z)

train_dataset = tf.data.Dataset.from_tensor_slices((train_x_through_2018.loc[:,
                                                    features_no_idx],
                                                    train_y_through_2018.loc[:, 'deaths'].values.reshape(-1,
                                                                                                         1))).repeat().shuffle(
    N)

m = OnOffSVGP(train_x_through_2018.loc[:, features_no_idx].values,
              train_y_through_2018.loc[:, 'deaths'].values.reshape(-1, 1)
              , kernf=f_kernel,
              kerng=g_kernel
              , likelihood=OnOffLikelihood()
              , Zf=Zf,
              Zg=Zg
              )

# fix the model noise term
m.likelihood.variance = 0.01
m.likelihood.variance.fixed = False

m.optimize(maxiter=iterations)  # ,method= tf.train.AdamOptimizer(learning_rate = 0.01)

pred_2019 = m.predict_onoffgp(x_just_2019.loc[:, features_no_idx].values)
pred_2019 = pred_2019[0]


AttributeError: 'numpy.ndarray' object has no attribute 'timesteps'

In [18]:
pred_2019

array([[0.06427505],
       [0.06427505],
       [0.06427505],
       ...,
       [0.02553089],
       [0.02553089],
       [0.02553089]])

In [21]:
single_time_pred

array([[0.06427505],
       [0.06796331],
       [0.05975341],
       ...,
       [0.02710086],
       [0.02543728],
       [0.02553089]])

In [28]:
pred_2019_df

grid_squar
25001010100    0.064275
25001010100    0.064275
25001010100    0.064275
25001010100    0.064275
25001010206    0.067963
                 ...   
25027761401    0.025437
25027761402    0.025531
25027761402    0.025531
25027761402    0.025531
25027761402    0.025531
Length: 6480, dtype: float64

In [31]:

pred_timesteps = x_just_2019[timestep_col].unique()


maes, top_10s, top_50s, top_100s = [], [], [], []
for timestep in pred_timesteps:
    single_time_pred = pred_2019[x_just_2019[timestep_col]==timestep]
    single_pred_df = pd.Series(single_time_pred.squeeze(), index=x_just_2019[x_just_2019[timestep_col]==timestep].grid_squar)
    single_time_true = y_just_2019[y_just_2019[timestep_col]==timestep]

    maes.append(mean_absolute_error(single_time_true.deaths, single_time_pred))
    top_10s.append(top_X(single_time_true.set_index('grid_squar')['deaths'], single_pred_df, 10))
    top_50s.append(top_X(single_time_true.set_index('grid_squar')['deaths'], single_pred_df, 50))
    top_100s.append(top_X(single_time_true.set_index('grid_squar')['deaths'], single_pred_df, 100))

model_fname = os.path.join(out_dir, 'model.pkl')
result_fname = os.path.join(out_dir, 'res.pkl')

m.savemodel(model_fname)
with open(result_fname, 'wb') as outfile:
    pickle.dump((maes, top_10s, top_50s, top_100s), outfile)

TypeError: Required argument 'file' (pos 2) not found

In [78]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x_through_2018.loc[:, ['lat','lon', 'qtr_since_2000', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile',]], 
                                                   train_y_through_2018.loc[:,'deaths'].values.reshape(-1,1))).repeat().shuffle(N)


In [36]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x_through_2018.loc[:, ['lat','lon', 'season_since_2000', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile', 'neighbors_last_timestep', 'last_timestep']], 
                                                   train_y_through_2018.loc[:,'deaths'].values.reshape(-1,1))).repeat().shuffle(N)


array([], dtype=float64)

In [79]:
features

['grid_squar',
 'lat',
 'lon',
 'qtr_since_2000',
 'theme_1_pc',
 'theme_2_pc',
 'theme_3_pc',
 'theme_4_pc',
 'svi_pctile']

In [39]:
train_x_through_2018

,grid_squar,lat,lon,qtr_since_2000,theme_1_pc,theme_2_pc,theme_3_pc,theme_4_pc,svi_pctile
0,25001010100,42.059829,-70.200407,0.0,0.7270,0.2972,0.5268,0.6872,0.6230
1,25001010100,42.059829,-70.200407,1.0,0.7270,0.2972,0.5268,0.6872,0.6230
2,25001010100,42.059829,-70.200407,2.0,0.7270,0.2972,0.5268,0.6872,0.6230
3,25001010100,42.059829,-70.200407,3.0,0.7270,0.2972,0.5268,0.6872,0.6230
4,25001010100,42.059829,-70.200407,4.0,0.7270,0.2972,0.5268,0.6872,0.6230
...,...,...,...,...,...,...,...,...,...
142543,25027761402,42.531342,-71.592751,71.0,0.1522,0.2139,0.3928,0.1570,0.1229
142544,25027761402,42.531342,-71.592751,72.0,0.1522,0.2139,0.3928,0.1570,0.1229
142545,25027761402,42.531342,-71.592751,73.0,0.1522,0.2139,0.3928,0.1570,0.1229
142546,25027761402,42.531342,-71.592751,74.0,0.1522,0.2139,0.3928,0.1570,0.1229


In [42]:
deaths_gdf_with_autoregressive.loc[train_x_through_2018.index][
    ['grid_squar', timestep_col, 'deaths']]

,grid_squar,qtr_since_2000,deaths
0,25001010100,0.0,NaN
1,25001010100,1.0,NaN
2,25001010100,2.0,NaN
3,25001010100,3.0,NaN
4,25001010100,4.0,NaN
...,...,...,...
142543,25027761402,71.0,NaN
142544,25027761402,72.0,NaN
142545,25027761402,73.0,NaN
142546,25027761402,74.0,NaN


In [60]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x_through_2018.loc[:,
                                                    features],
                                                    train_y_through_2018.loc[:, 'deaths'].values.reshape(-1,
                                                                                                         1))).repeat().shuffle(
    N)

TypeError: Expected binary or unicode string, got 42.0598291

In [73]:
tf.data.Dataset.from_tensor_slices((train_x_through_2018.loc[:,
                                                    features].values,
                                                    train_y_through_2018.loc[:, 'deaths'].values.reshape(-1,
                                                                                                         1)))

TypeError: Expected binary or unicode string, got 42.0598291

In [68]:
train_x_through_2018.loc[:,
                                                    'lat']

0         42.059829
1         42.059829
2         42.059829
3         42.059829
4         42.059829
            ...    
142543    42.531342
142544    42.531342
142545    42.531342
142546    42.531342
142547    42.531342
Name: lat, Length: 123120, dtype: float64

In [72]:
train_x_through_2018.loc[:,
                                                    features].values

array([['25001010100', 42.0598291, -70.2004073, ..., 0.5268,
        0.6872000000000001, 0.623],
       ['25001010100', 42.0598291, -70.2004073, ..., 0.5268,
        0.6872000000000001, 0.623],
       ['25001010100', 42.0598291, -70.2004073, ..., 0.5268,
        0.6872000000000001, 0.623],
       ...,
       ['25027761402', 42.5313417, -71.5927511, ..., 0.3928, 0.157,
        0.1229],
       ['25027761402', 42.5313417, -71.5927511, ..., 0.3928, 0.157,
        0.1229],
       ['25027761402', 42.5313417, -71.5927511, ..., 0.3928, 0.157,
        0.1229]], dtype=object)